In [1]:
from pynq import PL
PL.reset() #important fixes caching issues which have popped up.
from pynq import Overlay  #import the overlay module
ol = Overlay('./design_1.bit')  #locate/point to the bit file
import pprint

print(ol.ip_dict.keys())

dict_keys(['axi_dma_0', 'spi_adl_1', 'spi_lmx_0', 'spi_adl_0', 'usp_rf_data_converter_0', 'zynq_ultra_ps_e_0'])


In [2]:
adl1 = ol.spi_adl_0  
adl2 = ol.spi_adl_1 

def test_setup(): #tests that hardcoded values are being read out properly
    dead1 = adl1.read(0x00)  # read deadbeef hopefully (hard-coded in your mmio)
    zero1 = adl1.read(0x04)  # should be 0 hard coded
    dead2 = adl2.read(0x00)  # read deadbeef hopefully (hard-coded in your mmio)
    zero2 = adl2.read(0x04)  # should be 0 hard coded 
    if(hex(dead1)!="0xdeadbeef" or hex(zero1)!="0x0"):
        print("spi1 broken")
    if(hex(dead2)!="0xdeadbeef" or hex(zero2)!="0x0"):
        print("spi2 broken")

test_setup()

def read_only(): #read in spi input from all boards
    adl1.write(0x14,0x0) #should be trigger command
    d = adl1.read(0x08)  # should read the value of Spi input
    print("spi1: "+ hex(d)) 
    adl2.write(0x14,0x0) #should be trigger command
    d = adl2.read(0x08)  # should read the value of Spi input
    print("spi2: "+ hex(d))
    
def read_only_one(ip): #just read spi input from one of the modules
    ip.write(0x14,0x0) 
    d = ip.read(0x08)  
    print(hex(d)) 
    
def trigger_one(ip): #trigger only one module
    ip.write(0x14,0x0) 

def trigger(): #trigger both modules
    adl1.write(0x14,0x0)
    adl2.write(0x14,0x0)
    
def write_one(ip, command): #writes given command to one module
    ip.write(0x0C,command)
    ip.write(0x14,0x0)
    
def write(command): #writes given command to all modules 
    adl1.write(0x0C,command)  
    adl1.write(0x14,0x0)
    adl2.write(0x0C,command) 
    adl2.write(0x14,0x0) 

def write_and_read_one(ip, command): #writes given command to one module and reads incoming spi data
    ip.write(0x0C,command)  
    ip.write(0x14,0x0)
    d = ip.read(0x08)  
    print(hex(d))      
    
def write_and_read(command): #writes given command to all modules and reads incoming spi data
    adl1.write(0x0C,command)  
    adl1.write(0x14,0x0)
    d = adl1.read(0x08)  
    print(hex(d)) 
    adl2.write(0x0C,command) 
    adl2.write(0x14,0x0) 
    d = adl2.read(0x08)  
    print(hex(d)) 
    


In [154]:
#list of commands to input according to datasheet 
#first 4 chars are address, second 2 are actual input 


softreset = 0x0000_01
device_shutdown = 0x0002_02
un_device_shutdown = 0x0002_00

bypass_lo_config = 0x0021_00
blank_lo_config = 0x0020_00

lo_config = 0b0000_0000_0010_0000#_xx_xx  fill in these two to config


In [155]:
write_and_read_one(adl1, 0x1111_00)
write_and_read_one(adl2, 0x0000_11)

0xfffff
0xffe00


In [111]:
import time
import bitstring

MMIO_REGISTERS = {"TRIGGER": 0x14, "SPI_DATA_IN": 0x08, "SPI_DATA_OUT": 0x0C}

class LMX2595:
    def __init__(self, mmio_spi_controller):
        self.mmio_spi_controller = mmio_spi_controller
    
    def _spi_transaction(self, command):
        self.mmio_spi_controller.write(MMIO_REGISTERS["SPI_DATA_OUT"], command)  
        self.mmio_spi_controller.write(MMIO_REGISTERS["TRIGGER"], 0x0)
        time.sleep(0.1)
        return self.mmio_spi_controller.read(MMIO_REGISTERS["SPI_DATA_IN"])
    
    def _write_register(self, address, data):
        command = bitstring.BitArray(length = 24)
        command[0] = 0 # R/W bit
        command[1:8] = address
        command[8:] = data
        return self._spi_transaction(command.u)
    
    def _read_register(self, address):
        command = bitstring.BitArray(length = 24)
        command[0] = 1 # R/W bit
        command[1:8] = address
        command[8:] = 0 # Don't care
        return self._spi_transaction(command.u)

lmx = LMX2595(ol.spi_lmx_0)

In [135]:
lmx._write_register(0, 0b0010010000011000)

2868903935

In [136]:
print(hex(lmx._read_register(0)))

0xaaffffff


In [137]:
2 days agoprint(bin(lmx._read_register(0)))

0b10101010111111111111111111111111


In [131]:
print(bin(ol.spi_lmx_0.read(0x8))[8:])
print(bin(ol.spi_lmx_0.read(0x18))[8:])
print(bin(ol.spi_lmx_0.read(0x1C))[8:])

10111111111111111111111111
11111111111111111111111111
00111111111111111111111111


In [108]:
hex(2868903935)

'0xaaffffff'

In [33]:
import time
MMIO_REGISTERS = {"TRIGGER": 0x14, "SPI_DATA_IN": 0x08, "SPI_DATA_OUT": 0x0C}


soft_reset = 0x0000_81
device_shutdown = 0x0002_02
un_device_shutdown = 0x0002_00
lo_config = 0x0020_01
set_IF_filter = 0x0025_00
set_rgain = 0x0023_00
set_fgain = 0x0024_00


class ADL5960:
    def __init__(self, mmio_spi_controller):
        """
        mmio_spi_controller is a Pynq handle for an MMIO going to our custom SPI controller.
        initial_register_file is a path to a register map file exported from TICS-PRO for the default setup of the chip.
        """
        self.mmio_spi_controller = mmio_spi_controller
        self._spi_transaction(lo_config)
        self._spi_transaction(set_IF_filter)
        self._spi_transaction(set_rgain)
        self._spi_transaction(set_fgain)
        
    def _spi_transaction(self, command):
        self.mmio_spi_controller.write(MMIO_REGISTERS["SPI_DATA_OUT"], command)  
        self.mmio_spi_controller.write(MMIO_REGISTERS["TRIGGER"], 0x0)
        time.sleep(0.1)
        return self.mmio_spi_controller.read(MMIO_REGISTERS["SPI_DATA_IN"])
    
    def read_register(self,command):
        self.mmio_spi_controller.write(MMIO_REGISTERS["SPI_DATA_OUT"], (command+0x8000_00)) #add to command so that read bit is 1  
        self.mmio_spi_controller.write(MMIO_REGISTERS["TRIGGER"], 0x0)
        time.sleep(0.1)
        return self.mmio_spi_controller.read(MMIO_REGISTERS["SPI_DATA_IN"])
    
    def shutdown(self):
        self._spi_transaction(device_shutdown)
    
    def softreset(self):
        self._spi_transaction(soft_reset)
    def unshutdown(self):
        self._spi_transaction(un_device_shutdown)
        
    
    
adl1 = ADL5960(ol.spi_adl_0)  
adl2 = ADL5960(ol.spi_adl_1) 